In [2]:
import numpy as np
import pandas as pd

def hilbert_exp_adjust_weights(weights, reward_signal, eta=0.01):
    log_update = np.log(weights + 1e-12) + eta * reward_signal
    exp_update = np.exp(log_update - np.max(log_update))
    exp_update = np.maximum(exp_update, 1e-12)
    exp_update /= np.sum(exp_update)
    return exp_update

# Generate synthetic data
np.random.seed(42)
dates = pd.date_range("2010-01-01", periods=2520, freq='B')
assets = ["AAPL", "MSFT", "GOOG", "AMZN", "META"]
returns = pd.DataFrame(0.001 + 0.02 * np.random.randn(len(dates), len(assets)), index=dates, columns=assets)

# Functions for tests
def test_weight_normalization(weights_hist):
    for w in weights_hist:
        assert np.abs(np.sum(w) - 1) < 1e-8, f"Normalization failed: sum={np.sum(w)}"
    print("✅ Weight normalization test passed.")

def test_positivity(weights_hist):
    for w in weights_hist:
        assert np.all(w > 0), "Positivity test failed: negative or zero weights found."
    print("✅ Positivity test passed.")

def test_convergence_stationary():
    reward_signal = np.array([0.05, 0.01, 0.01, 0.01, 0.01])
    weights = np.ones(5) / 5
    for _ in range(3000):  # Increase iterations
        weights = hilbert_exp_adjust_weights(weights, reward_signal, eta=0.5)  # Increase eta
    assert weights[0] > 0.95, f"Convergence failed: final weights = {weights}"
    print("✅ Convergence to dominant asset test passed.")

def test_regret(returns, weights_hist):
    cum_rewards = (returns.iloc[60:].values * weights_hist).sum(axis=1).cumsum()
    single_asset_rewards = returns.iloc[60:].cumsum()
    best_asset = single_asset_rewards.iloc[-1].idxmax()
    best_cum = single_asset_rewards[best_asset].iloc[-1]
    algo_cum = cum_rewards[-1]
    regret = best_cum - algo_cum
    print(f"✅ Regret test completed. Regret = {regret:.4f}")

def test_eta_sensitivity(returns):
    etas = [0.001, 0.01, 0.1, 0.5]
    for eta in etas:
        weights = np.ones(5) / 5
        for t in range(60, 300):
            reward_vector = returns.iloc[t].values
            weights = hilbert_exp_adjust_weights(weights, reward_vector, eta=eta)
        print(f"✅ Eta sensitivity test passed for eta={eta}. Final weights: {weights}")

def test_robustness_noise(returns):
    weights = np.ones(5) / 5
    for t in range(60, 300):
        reward_vector = returns.iloc[t].values + np.random.normal(0, 0.02, 5)
        weights = hilbert_exp_adjust_weights(weights, reward_vector, eta=0.01)
    assert np.all(weights > 0), "Robustness test failed: zero or negative weights."
    print("✅ Robustness to noise test passed.")

def test_cross_validation(returns):
    mid_point = len(returns) // 2
    weights1 = np.ones(5) / 5
    for t in range(60, mid_point):
        reward_vector = returns.iloc[t].values
        weights1 = hilbert_exp_adjust_weights(weights1, reward_vector, eta=0.01)
    print(f"✅ Cross-validation: first half final weights: {weights1}")
    weights2 = weights1.copy()
    for t in range(mid_point, len(returns)):
        reward_vector = returns.iloc[t].values
        weights2 = hilbert_exp_adjust_weights(weights2, reward_vector, eta=0.01)
    print(f"✅ Cross-validation: second half final weights: {weights2}")

def test_path_dependence(returns):
    weights = np.ones(5) / 5
    order = np.arange(60, 300)
    np.random.shuffle(order)
    for t in order:
        reward_vector = returns.iloc[t].values
        weights = hilbert_exp_adjust_weights(weights, reward_vector, eta=0.01)
    print(f"✅ Path dependence test: final weights after shuffled order: {weights}")

def test_perturbation_effect(returns):
    base_weights = np.ones(5) / 5
    perturbed_weights = base_weights + np.random.normal(0, 0.001, 5)
    perturbed_weights /= np.sum(perturbed_weights)
    for t in range(60, 300):
        reward_vector = returns.iloc[t].values
        base_weights = hilbert_exp_adjust_weights(base_weights, reward_vector, eta=0.01)
        perturbed_weights = hilbert_exp_adjust_weights(perturbed_weights, reward_vector, eta=0.01)
    diff = np.linalg.norm(base_weights - perturbed_weights)
    print(f"✅ Perturbation test: norm difference in final weights = {diff:.6f}")

# Run main training to generate weights history
weights = np.ones(5) / 5
weights_history = []
for t in range(60, len(returns)):
    reward_vector = returns.iloc[t].values
    weights = hilbert_exp_adjust_weights(weights, reward_vector, eta=0.01)
    weights_history.append(weights)

weights_history = np.array(weights_history)

# Run tests
test_weight_normalization(weights_history)
test_positivity(weights_history)
test_convergence_stationary()
test_regret(returns, weights_history)
test_eta_sensitivity(returns)
test_robustness_noise(returns)
test_cross_validation(returns)
test_path_dependence(returns)
test_perturbation_effect(returns)

print("🎉🎉 All primary theoretical and numerical tests completed! 🎉🎉")


✅ Weight normalization test passed.
✅ Positivity test passed.
✅ Convergence to dominant asset test passed.
✅ Regret test completed. Regret = 1.6707
✅ Eta sensitivity test passed for eta=0.001. Final weights: [0.19993616 0.19992714 0.19998599 0.20003754 0.20011317]
✅ Eta sensitivity test passed for eta=0.01. Final weights: [0.19936146 0.19927156 0.19985883 0.20037469 0.20113346]
✅ Eta sensitivity test passed for eta=0.1. Final weights: [0.19360124 0.19272997 0.19848578 0.20366887 0.21151413]
✅ Eta sensitivity test passed for eta=0.5. Final weights: [0.16791489 0.1641704  0.19019343 0.21635744 0.26136384]
✅ Robustness to noise test passed.
✅ Cross-validation: first half final weights: [0.19988207 0.19918634 0.19992232 0.19905697 0.2019523 ]
✅ Cross-validation: second half final weights: [0.19991011 0.20115486 0.19788806 0.19766938 0.20337759]
✅ Path dependence test: final weights after shuffled order: [0.19936146 0.19927156 0.19985883 0.20037469 0.20113346]
✅ Perturbation test: norm diff